In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import xmltodict as xd

In [108]:
# #open and convert the xml to csv
# with open('data/mochada_mpesa.xml', 'r') as file:
#     data = xd.parse(file.read())


# #extracting the messages
# messages = data['smses']['sms']

# #converting to dataframe
# df = pd.DataFrame(messages)


# #save as a csv(optional)
# df.to_csv('mochada_sms.csv', index=False)

# df.tail()

In [109]:
"""  
- create a new dataframe with only three main columns; Address, Body, and readable date
- change readable date to pandas datetime object
- inspect and see what information to extract from the body ie balance amount, amount taken or given
- perform a weekly analysis from Wednesday to Tuesday
- calculate the monthly commission earned based on the transactions
- Do a time series analysis of the dataset
"""

'  \n- create a new dataframe with only three main columns; Address, Body, and readable date\n- change readable date to pandas datetime object\n- inspect and see what information to extract from the body ie balance amount, amount taken or given\n- perform a weekly analysis from Wednesday to Tuesday\n- calculate the monthly commission earned based on the transactions\n- Do a time series analysis of the dataset\n'

In [110]:
df = pd.read_csv('mochada_sms.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   @protocol        3080 non-null   int64  
 1   @address         3080 non-null   object 
 2   @date            3080 non-null   int64  
 3   @type            3080 non-null   int64  
 4   @subject         0 non-null      float64
 5   @body            3080 non-null   object 
 6   @toa             0 non-null      float64
 7   @sc_toa          0 non-null      float64
 8   @service_center  3080 non-null   int64  
 9   @read            3080 non-null   int64  
 10  @status          3080 non-null   int64  
 11  @locked          3080 non-null   int64  
 12  @date_sent       3080 non-null   int64  
 13  @sub_id          3080 non-null   int64  
 14  @readable_date   3080 non-null   object 
 15  @contact_name    3080 non-null   object 
dtypes: float64(3), int64(9), object(4)
memory usage: 385.1+ KB


In [111]:
df.head()

,@protocol,@address,@date,@type,@subject,@body,@toa,@sc_toa,@service_center,@read,@status,@locked,@date_sent,@sub_id,@readable_date,@contact_name
0,0,MPESA,1713512313950,1,NaN,SDJ7N53FS3 Confirmed. On 19/4/24 at 10:38 AM T...,NaN,NaN,254722500040,1,-1,0,1713512297000,2,"Apr 19, 2024 10:38:33 AM",(Unknown)
1,0,MPESA,1713512587737,1,NaN,SDJ9N5M21L Confirmed. On 19/4/24 at 10:42 AM T...,NaN,NaN,254722500040,1,-1,0,1713512571000,2,"Apr 19, 2024 10:43:07 AM",(Unknown)
2,0,MPESA,1713512589079,1,NaN,SDJ9MU5DIL Confirmed. On 19/4/24 at 8:57 AM Ta...,NaN,NaN,254722500166,1,-1,0,1713506260000,2,"Apr 19, 2024 10:43:09 AM",(Unknown)
3,0,MPESA,1713512590679,1,NaN,SDJ2MYL302 Confirmed. On 19/4/24 at 9:39 AM Ta...,NaN,NaN,254722500040,1,-1,0,1713508798000,2,"Apr 19, 2024 10:43:10 AM",(Unknown)
4,0,MPESA,1713512593038,1,NaN,SDJ4N12JZY Confirmed. On 19/4/24 at 10:03 AM T...,NaN,NaN,254722500166,1,-1,0,1713510188000,2,"Apr 19, 2024 10:43:13 AM",(Unknown)


In [112]:
#columns to keep
data = df[['@address', '@body', '@readable_date']]


#renaming the columns 
data.columns = ['name', 'body', 'date']

#converting the date into datetime object
data['date'] = pd.to_datetime(data['date'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   name    3080 non-null   object        
 1   body    3080 non-null   object        
 2   date    3080 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 72.3+ KB


<ipython-input-112-c51030f18ed6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])


In [113]:
data.head(10)

,name,body,date
0,MPESA,SDJ7N53FS3 Confirmed. On 19/4/24 at 10:38 AM T...,2024-04-19 10:38:33
1,MPESA,SDJ9N5M21L Confirmed. On 19/4/24 at 10:42 AM T...,2024-04-19 10:43:07
2,MPESA,SDJ9MU5DIL Confirmed. On 19/4/24 at 8:57 AM Ta...,2024-04-19 10:43:09
3,MPESA,SDJ2MYL302 Confirmed. On 19/4/24 at 9:39 AM Ta...,2024-04-19 10:43:10
4,MPESA,SDJ4N12JZY Confirmed. On 19/4/24 at 10:03 AM T...,2024-04-19 10:43:13
5,MPESA,SDJ4N6KYGK Confirmed. On 19/4/24 at 10:51 AM T...,2024-04-19 10:51:30
6,MPESA,SDJ3NAIRL9 confirmed.You bought Ksh50.00 of ai...,2024-04-19 11:25:45
7,MPESA,SDJ4NEIHH0 Confirmed. On 19/4/24 at 11:59 AM T...,2024-04-19 11:59:58
8,MPESA,SDJ6NF0IHS Confirmed. On 19/4/24 at 12:03 PM T...,2024-04-19 12:04:14
9,MPESA,SDJ2NG9PAO Confirmed. On 19/4/24 at 12:14 PM T...,2024-04-19 12:14:52


In [114]:
data.tail(15)

,name,body,date
3065,MPESA,SGQ6Y1YOHQ Confirmed. Ksh400.00 sent to Equity...,2024-07-26 21:33:31
3066,MPESA,SGR12163XV Confirmed. Ksh100.00 sent to CLAIRE...,2024-07-27 18:24:19
3067,MPESA,SGS83T5DGW Confirmed. On 28/7/24 at 8:58 AM Ta...,2024-07-28 08:59:11
3068,MPESA,SGS43U9NDQ Confirmed. On 28/7/24 at 9:10 AM Ta...,2024-07-28 09:10:15
3069,MPESA,SGS03XREVC Confirmed. On 28/7/24 at 9:43 AM Ta...,2024-07-28 09:43:46
3070,MPESA,SGS33XYP3F Confirmed. on 28/7/24 at 9:45 AM Gi...,2024-07-28 09:45:25
3071,MPESA,SGS33YLZY1 Confirmed. on 28/7/24 at 9:51 AM Gi...,2024-07-28 09:51:32
3072,MPESA,SGS03YVQME Confirmed. On 28/7/24 at 9:54 AM Ta...,2024-07-28 09:54:29
3073,MPESA,SGS63Z7FII Confirmed. on 28/7/24 at 9:57 AM Gi...,2024-07-28 09:57:13
3074,MPESA,SGS444M4UK Confirmed. on 28/7/24 at 10:46 AM G...,2024-07-28 10:46:49


In [115]:
data['body'].iloc[0]

'SDJ7N53FS3 Confirmed. On 19/4/24 at 10:38 AM Take Ksh1,100.00 cash from ROBERT MAHINDA Your M-PESA float balance is Ksh82,007.00. Click the link to Download M-Pesa Agent App and Transact the SMART way https://bit.ly/3Ll6JQU'

The `body` column seems to contain a lot of information, lets write a function to help us extract them and use them for analysis.

In [116]:
#lets create a function to extract information from this column
""" 
Function takes the a sting as input
Strips it into different sections
Extracts specific information from the texts;
    - unique id
    - transaction status
    - transaction type
    - amount
    - customer names
    - balance
"""

import re 

def extract(text):
    #split the text
    parts = text.split()
    
    #initialize variables
    id = parts[0]
    status = parts[1]

        #determine the actions
    if 'Take' in parts:#someone gave money to deposit
        action = 'Take'
        action_index = parts.index('Take')

    elif 'Give' in parts:
        action = 'Give'#someone withdrew money
        action_index = parts.index('Give')

    elif 'bought' in parts:
        action = 'bought'#sold airtime to customer
        action_index = parts.index('bought')

    elif 'sent' in parts:
        action = 'sent'#personal transactions
        action_index = parts.index('sent')

    else:
        action = None
        action_index = -1

    #extraction of the amount 
    amount_pattern = r'Ksh[\d,]+\.\d{2}'
    amount_match = re.search(amount_pattern, text)
    amount = amount_match.group(0) if amount_match else None
    # amount = parts[action_index + 1] if action_index != -1 else None

    #extraction of customer name
    if action in ['Take', 'Give']:
        name = parts[action_index + 4] + '' + parts[action_index + 5]
    else:
        name = None

    #extraction of mpesa balance
    bl_start = text.find('balance is') + len('balance is') + 1
    bl_end = text.find('Ksh', bl_start ) + text[bl_start:].find('Kshs') + len('Kshs') + text[bl_start:].find('.') + 3
    balance = text[bl_start:bl_end].strip() if bl_start != -1 and bl_end != -1 else None

    return {
        'Transaction Id':id,
        'Status':status,
        'Action':action,
        'Amount':amount,
        'Customer Name':name,
        'balance':balance,
    }


In [117]:
#lets apply the formula to the 'body' column
new_df = data['body'].apply(extract).apply(pd.Series)

new_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Transaction Id  3080 non-null   object
 1   Status          3080 non-null   object
 2   Action          2725 non-null   object
 3   Amount          2891 non-null   object
 4   Customer Name   2491 non-null   object
 5   balance         3080 non-null   object
dtypes: object(6)
memory usage: 144.5+ KB


In [118]:
new_df.head()

,Transaction Id,Status,Action,Amount,Customer Name,balance
0,SDJ7N53FS3,Confirmed.,Take,"Ksh1,100.00",ROBERTMAHINDA,"Ksh82,007.00. C"
1,SDJ9N5M21L,Confirmed.,Take,"Ksh1,100.00",brianchacha,"Ksh80,907.00. C"
2,SDJ9MU5DIL,Confirmed.,Take,"Ksh1,100.00",ChristopherRwara,"Ksh97,007.00. C"
3,SDJ2MYL302,Confirmed.,Take,Ksh100.00,johntimpanko,"Ksh83,407.00. C"
4,SDJ4N12JZY,Confirmed.,Take,Ksh300.00,NICKSONMIDUNGA,"Ksh83,107.00. C"


In [119]:
new_df['balance'].unique().sum()

"Ksh82,007.00. CKsh80,907.00. CKsh97,007.00. CKsh83,407.00. CKsh83,107.00. CKsh80,807.00. CKsh738.04. TKsh80,307.00. CKsh76,637.00. CKsh76,587.00. CKsh71,087.00. Cndly capture the correct mobile number and customer details as they appear on the identification document and attempt the depositKsh70,849.00. CKsh70,287.00. CKsh69,801.00. CKsh68,801.00. CKsh68,751.00. CKsh69,751.00. ClKsh68,951.00. CKsh66,951.00. CKsh66,713.00. CKsh66,501.00. CKsh65,701.00. CKsh65,381.00. CKsh65,281.00. CKsh65,081.00. CKsh64,231.00. CKsh63,160.00. CKsh63,042.00. CKsh58,042.00. CKsh57,942.00. CKsh59,242.00. ClKsh58,585.00. CKsh54,427.00. CKsh50,940.00. CKsh46,035.00. CKsh44,235.00. CKsh44,082.00. CKsh43,105.00. CKsh38,905.00. CKsh38,310.00. CKsh38,260.00. CKsh38,210.00. CKsh38,010.00. CKsh38,510.00. ClKsh38,160.00. CKsh6,338.04. UKsh38,110.00. CKsh37,960.00. CKsh36,711.00. CKsh39,211.00. ClKsh39,161.00. CKsh39,043.00. CKsh38,953.00. CKsh38,853.00. CKsh36,353.00. CKsh36,653.00. ClKsh32,653.00. CKsh32,583.00. 

In [120]:
import re

def extract_info(text):
    # Define the regular expression patterns for the different pieces of information
    transaction_id_pattern = r'^[A-Z0-9]+'
    status_pattern = r'(Confirmed|confirmed|You bought)'
    action_pattern = r'(Take|Give|bought)'
    amount_pattern = r'Ksh[\d,]+\.\d{2}'
    name_pattern = r'\b[A-Z][a-zA-Z]*\s[A-Z][a-zA-Z]*\b'
    balance_type_pattern = r'(float|balance)'
    balance_pattern = r'Ksh[\d,]+\.\d{2}'

    # Extract using the patterns
    transaction_id = re.search(transaction_id_pattern, text).group(0)
    status_match = re.search(status_pattern, text)
    status = status_match.group(0) if status_match else None
    action_match = re.search(action_pattern, text)
    action = action_match.group(0) if action_match else None
    amount_match = re.search(amount_pattern, text)
    amount = amount_match.group(0) if amount_match else None

    # Extract the name only if the action is "Take" or "Give"
    recipient_name = None
    if action in ['Take', 'Give', 'bought', 'sent']:
        names = re.findall(name_pattern, text)
        recipient_name = names[-1] if names else None

    balance_type_match = re.search(balance_type_pattern, text)
    balance_type = balance_type_match.group(0) if balance_type_match else None

    # Extract the last balance amount
    balances = re.findall(balance_pattern, text)
    balance = balances[-1] if balances else None

    return {
        'Transaction ID': transaction_id,
        'Status': status,
        'Action': action,
        'Amount': amount,
        'Recipient Name': recipient_name,
        'Balance Type': balance_type,
        'Balance': balance
    }


# Apply the extraction function to the 'messages' column
df_extracted = data['body'].apply(extract_info).apply(pd.Series)
# joined_df = pd.concat([df_extracted, data], join= 'inner')

# joined_df.info()


In [104]:
# joined_df.tail()

,Transaction ID,Status,Action,Amount,Recipient Name,Balance Type,Balance,name,body,date
3075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MPESA,SGS245L5GQ Confirmed. On 28/7/24 at 10:55 AM T...,2024-07-28 10:55:57
3076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MPESA,SGS8488HBK Confirmed. on 28/7/24 at 11:20 AM G...,2024-07-28 11:20:05
3077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MPESA,SGS449HBC6 Confirmed. On 28/7/24 at 11:31 AM T...,2024-07-28 11:31:31
3078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MPESA,SGS949ZVIH Confirmed. On 28/7/24 at 11:35 AM T...,2024-07-28 11:36:22
3079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MPESA,SGS04DA44U Confirmed. On 28/7/24 at 12:05 PM T...,2024-07-28 12:05:47


In [78]:
df_extracted['Balance Type'].unique()

array(['float', 'balance', None], dtype=object)